<a href="https://colab.research.google.com/github/Dilpreetk11/Dilpreetk11/blob/main/twitter_sentiment_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers torch gradio pandas scikit-learn

import pandas as pd
import numpy as np
import re
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
import torch
import gradio as gr


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 93.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 93.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 55.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 95.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [2]:
df = pd.read_csv("Tweets.csv")[['text', 'airline_sentiment']]
df = df.dropna(subset=['text', 'airline_sentiment'])
df.head()


,text,airline_sentiment
0,@VirginAmerica What @dhepburn said.,neutral
1,@VirginAmerica plus you've added commercials t...,positive
2,@VirginAmerica I didn't today... Must mean I n...,neutral
3,@VirginAmerica it's really aggressive to blast...,negative
4,@VirginAmerica and it's a really big bad thing...,negative


In [3]:
def clean_text(text):
    text = re.sub(r"http\S+", "", text)             # Remove URLs
    text = re.sub(r"@\w+", "", text)                # Remove mentions
    text = re.sub(r"#", "", text)                   # Remove hashtag symbol
    text = re.sub(r"[^\w\s']", ' ', text)           # Remove punctuation (except apostrophes)
    text = re.sub(r"\d+", "", text)                 # Remove numbers
    text = text.lower().strip()
    text = re.sub('\s+', ' ', text)                 # Remove extra spaces
    return text

df['clean_text'] = df['text'].apply(clean_text)


In [4]:
# We'll do binary: negative=1, else=0 (adjust for multiclass if you want)
df['label'] = df['airline_sentiment'].apply(lambda x: 1 if x == 'negative' else 0)


In [5]:
from sklearn.model_selection import train_test_split

train_texts, val_texts, train_labels, val_labels = train_test_split(
    df['clean_text'].tolist(),
    df['label'].tolist(),
    test_size=0.2,
    stratify=df['label'],
    random_state=42
)


In [6]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
max_length = 128

def tokenize_texts(texts):
    return tokenizer(
        texts,
        padding=True,
        truncation=True,
        max_length=max_length,
        return_tensors='pt'
    )

train_encodings = tokenize_texts(train_texts)
val_encodings = tokenize_texts(val_texts)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [7]:
import torch

class TweetDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels
    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx], dtype=torch.long)
        return item
    def __len__(self):
        return len(self.labels)

train_dataset = TweetDataset(train_encodings, train_labels)
val_dataset = TweetDataset(val_encodings, val_labels)


In [9]:
from transformers import BertForSequenceClassification, Trainer, TrainingArguments

model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=2)

training_args = TrainingArguments(
    output_dir='./results',
    num_train_epochs=2,
    per_device_train_batch_size=16,       # Increased for better GPU usage
    per_device_eval_batch_size=16,
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_dir='./logs',
    logging_steps=100,                    # Optional: reduce logging frequency
    load_best_model_at_end=True,
    fp16=True,                            # Enable mixed precision on T4
    report_to=["none"]                    # Disable external logging
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

trainer.train()


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch,Training Loss,Validation Loss
1,0.328900,0.291389
2,0.168700,0.359576


TrainOutput(global_step=1464, training_loss=0.27871540139933104, metrics={'train_runtime': 220.7926, 'train_samples_per_second': 106.091, 'train_steps_per_second': 6.631, 'total_flos': 529642554439680.0, 'train_loss': 0.27871540139933104, 'epoch': 2.0})

In [10]:
import numpy as np
from sklearn.metrics import classification_report, accuracy_score

preds = trainer.predict(val_dataset)
pred_labels = np.argmax(preds.predictions, axis=1)
print(classification_report(val_labels, pred_labels))
print("Accuracy:", accuracy_score(val_labels, pred_labels))


              precision    recall  f1-score   support

           0       0.83      0.86      0.84      1092
           1       0.91      0.90      0.91      1836

    accuracy                           0.88      2928
   macro avg       0.87      0.88      0.88      2928
weighted avg       0.88      0.88      0.88      2928

Accuracy: 0.8828551912568307


In [12]:
model.save_pretrained('bert_airline_sentiment')
tokenizer.save_pretrained('bert_airline_sentiment')


('bert_airline_sentiment/tokenizer_config.json',
 'bert_airline_sentiment/special_tokens_map.json',
 'bert_airline_sentiment/vocab.txt',
 'bert_airline_sentiment/added_tokens.json')

In [13]:
from transformers import BertForSequenceClassification, BertTokenizer
import torch

model = BertForSequenceClassification.from_pretrained('bert_airline_sentiment')
tokenizer = BertTokenizer.from_pretrained('bert_airline_sentiment')


In [14]:
import gradio as gr

def clean_text(text):
    import re
    text = re.sub(r"http\S+", "", text)
    text = re.sub(r"@\w+", "", text)
    text = re.sub(r"#", "", text)
    text = re.sub(r"[^\w\s']", ' ', text)
    text = re.sub(r"\d+", "", text)
    text = text.lower().strip()
    text = re.sub('\s+', ' ', text)
    return text

def classify_sentiment(tweet):
    cleaned = clean_text(tweet)
    encoding = tokenizer(cleaned, return_tensors='pt', truncation=True, padding=True, max_length=128)
    with torch.no_grad():
        output = model(**encoding)
        pred = torch.argmax(output.logits, dim=1).item()
    return "Negative" if pred == 1 else "Not Negative"

gr.Interface(
    fn=classify_sentiment,
    inputs=gr.Textbox(lines=2, label="Enter a tweet"),
    outputs="text",
    title="BERT Twitter Sentiment Classifier",
    description="Enter a tweet to see if it's negative or not."
).launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://baee225f0b14915f01.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [19]:
# Zip saved folder
import shutil
shutil.make_archive('bert_airline_sentiment_model', 'zip', 'bert_airline_sentiment')

# For Colab only: trigger download
from google.colab import files
files.download('bert_airline_sentiment_model.zip')


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [18]:
import zipfile
import os

def zipdir(path, ziph):
    # ziph is zipfile handle
    for root, dirs, files in os.walk(path):
        for file in files:
            ziph.write(os.path.join(root, file),
                       os.path.relpath(os.path.join(root, file),
                                       os.path.join(path, '..')))

with zipfile.ZipFile('bert_airline_sentiment_model.zip', 'w', zipfile.ZIP_DEFLATED) as zipf:
    zipdir('bert_airline_sentiment', zipf)